# 🧪 Test de généralisation des modèles entraînés à Lille sur les données de Bordeaux

Ce notebook a pour objectif d’évaluer la capacité de généralisation des modèles de prédiction de prix au m², initialement entraînés sur les logements de Lille (2022), lorsqu'ils sont appliqués sur les logements de Bordeaux (2022).

## 🗂️ Contexte et objectifs
Dans le notebook précédent (`test_model_lille.ipynb`), plusieurs modèles ont été entraînés et comparés pour deux types de biens :

- 🏠 Maisons
- 🏢 Appartements

Les meilleurs modèles ont été sauvegardés sous forme de pipelines complets (prétraitement + régression).

L’objectif ici est de :
1. Charger ces pipelines sauvegardés.
2. Appliquer exactement les mêmes étapes de nettoyage et de préparation sur les données de Bordeaux.
3. Évaluer leurs performances sur Bordeaux.
4. Identifier quels modèles généralisent le mieux.
5. (Optionnel) Comparer leurs performances avec celles obtenues sur Lille.

## 🧼 Nettoyage des données de Bordeaux
Le nettoyage appliqué dans ce fichier est identique à celui effectué pour Lille :

|Étape|	Description|
|---|---|
|Suppression des doublons|	Suppression des logements dupliqués
|Suppression des valeurs aberrantes|	Filtrage des surfaces et prix hors normes
|Filtrage par type de bien|	Séparation entre maisons et appartements
|Calcul du prix au m²|	Création d’une colonne prix_m2 = prix / surface
|Sélection des variables|	Choix de variables explicatives pertinentes (différentes pour maison/appartement)

Cela permet d'assurer que les données de Bordeaux soient préparées de manière cohérente, pour une comparaison équitable avec les modèles formés sur Lille.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

In [ ]:
# Chargement depuis parquet
df = pd.read_parquet("../data/clean/bordeaux_2022.parquet")

# Filtrer biens avec exactement 4 pièces principales
df_4p = df[df['Nombre pieces principales'] == 4].copy()

In [ ]:
colonnes_a_garder = [
    'Surface reelle bati',
    'Nombre pieces principales',
    'Nombre de lots',
    'Valeur fonciere',
    'Surface terrain',
    'Code type local',
    'prix_m2'
]

# logements
df_logements = df_4p[colonnes_a_garder].copy()

In [ ]:
# Remplacer Surface terrain NaN par 0 pour les appartements (Code type local == 2)
df_logements.loc[
    (df_logements["Code type local"] == 2) & (df_logements["Surface terrain"].isna()),
    "Surface terrain"
] = 0

# Remplacer Surface terrain NaN par 0 pour les maisons (Code type local == 1)
df_logements.loc[
    (df_logements["Code type local"] == 1) & (df_logements["Surface terrain"].isna()),
    "Surface terrain"
] = 0

df_logements = df_logements.dropna()

print(len(df_logements))

In [ ]:
#séparation du dataset
df_bx_maisons_raw = df_logements[df_logements["Code type local"] == 1]
df_bx_apt_raw = df_logements[df_logements["Code type local"] == 2]

In [ ]:
def detect_outliers(df, column):
    
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    return lower_bound, upper_bound, outliers

In [ ]:
def remove_outliers(df, column):
    lower, upper, _ = detect_outliers(df, column)
    return df[(df[column] >= lower) & (df[column] <= upper)]

def median_outliers(df, column):
    lower, upper, _ = detect_outliers(df, column)
    mediane = df[column].median()
    df.loc[(df[column] < lower) | (df[column] > upper), column] = mediane
    return df

In [ ]:
df_bx_maisons_clean = median_outliers(df_bx_maisons_raw, "prix_m2")
df_bx_apt_clean = median_outliers(df_bx_apt_raw, "prix_m2")

df_bx_maisons_clean = median_outliers(df_bx_maisons_raw, "Nombre de lots")
df_bx_apt_clean = median_outliers(df_bx_apt_raw, "Nombre de lots")

df_bx_maisons_clean = median_outliers(df_bx_maisons_raw, "Surface terrain")
df_bx_apt_clean = median_outliers(df_bx_apt_raw, "Surface terrain")

df_bx_maisons_clean = median_outliers(df_bx_maisons_raw, "Surface reelle bati")
df_bx_apt_clean = median_outliers(df_bx_apt_raw, "Surface reelle bati")

print(len(df_bx_maisons_clean))
print(len(df_bx_apt_clean))

In [ ]:
# Créer la figure et les 2 sous-graphiques côte à côte
fig, axs = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

# --- 1. Histogramme pour les maisons ---
axs[0].hist(df_bx_maisons_clean['prix_m2'], bins=30, color='orange', edgecolor='w')
axs[0].axvline(np.median(df_bx_maisons_clean['prix_m2']), color='red', linestyle='--', label='Médiane')
axs[0].set_title("Maisons - prix_m2")
axs[0].set_xlabel("prix_m2")
axs[0].set_ylabel("Nombre de biens")
axs[0].grid(True)
axs[0].legend()

# --- 2. Histogramme pour les appartements ---
axs[1].hist(df_bx_apt_clean['prix_m2'], bins=30, color='orange', edgecolor='w')
axs[1].axvline(np.median(df_bx_apt_clean['prix_m2']), color='red', linestyle='--', label='Médiane')
axs[1].set_title("Appartements - prix_m2")
axs[1].set_xlabel("prix_m2")
axs[1].grid(True)
axs[1].legend()

# Ajuster l'espacement
plt.tight_layout()
plt.show()

In [ ]:
# Pour les maisons
df_bx_maisons = df_bx_maisons_clean[(df_bx_maisons_clean["prix_m2"] >= 1500) & (df_bx_maisons_clean["prix_m2"] <= 5000)]

# Pour les appartements
df_bx_apt = df_bx_apt_clean[(df_bx_apt_clean["prix_m2"] >= 2000) & (df_bx_apt_clean["prix_m2"] <= 7000)]

In [ ]:
# Créer la figure et les 2 sous-graphiques côte à côte
fig, axs = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

# --- 1. Histogramme pour les maisons ---
axs[0].hist(df_bx_maisons['prix_m2'], bins=30, color='orange', edgecolor='w')
axs[0].axvline(np.median(df_bx_maisons['prix_m2']), color='red', linestyle='--', label='Médiane')
axs[0].set_title("Maisons - prix_m2")
axs[0].set_xlabel("prix_m2")
axs[0].set_ylabel("Nombre de biens")
axs[0].grid(True)
axs[0].legend()

# --- 2. Histogramme pour les appartements ---
axs[1].hist(df_bx_apt['prix_m2'], bins=30, color='orange', edgecolor='w')
axs[1].axvline(np.median(df_bx_apt['prix_m2']), color='red', linestyle='--', label='Médiane')
axs[1].set_title("Appartements - prix_m2")
axs[1].set_xlabel("prix_m2")
axs[1].grid(True)
axs[1].legend()

# Ajuster l'espacement
plt.tight_layout()
plt.show()

## 🏗️ Chargement des pipelines sauvegardés
On charge les pipelines d’apprentissage automatique précédemment entraînés sur les données de Lille (en 2022), séparément pour les appartements et les maisons. Ces pipelines contiennent à la fois le prétraitement des données et le modèle de régression final.

In [ ]:
# Charger la pipeline sauvegardée (ex : pour les appartements)
pipeline_appartement = joblib.load("../models/pipeline_appartement_models.pkl")

# Pour les maisons (si tu l'as aussi sauvegardée)
pipeline_maison = joblib.load("../models/pipeline_maison_models.pkl")

print(type(pipeline_appartement))

## Préparer les données pour l'entraînement

## 🧮 Sélection des variables explicatives

On selectionnes les variables explicatives (features) qui seront utilisées pour faire les prédictions du prix au m² avec les pipelines.

Ces variables sont choisies pour correspondre à celles utilisées lors de l’entraînement des modèles sur Lille, afin de garantir une compatibilité parfaite avec les pipelines.

In [ ]:
# Variables explicatives
features_maison = ["Surface terrain", "Surface reelle bati"]
features_appartement = ["Nombre de lots", "Surface reelle bati"]

#maison
X_bx_maisons = df_bx_maisons[features_maison]
y_bx_maisons = df_bx_maisons["prix_m2"]

# appartement
X_bx_apt = df_bx_apt[features_appartement]
y_bx_apt = df_bx_apt["prix_m2"]

## 📈 Évaluation comparative des modèles par pipeline

Chaque pipeline contient plusieurs modèles entraînés (par exemple : LinearRegression, RandomForest, etc.).
On évalue un à un pour identifier celui qui généralise le mieux sur les données de Bordeaux.

### 🧪 Fonction d’évaluation affiche_score
Cette fonction prend en entrée un dictionnaire de modèles (dans le pipeline), les données d'entrée X_bx, et les valeurs cibles y_bx.

Pour chaque modèle, elle calcule les métriques suivantes :

|Métrique|	Description
|---|---
|MSE|	Erreur quadratique moyenne
|RMSE|	Racine de l'erreur quadratique moyenne
|MAE|	Erreur absolue moyenne
|R²|	Score de détermination (qualité de la prédiction)

### 🏆 Classement des modèles
On applique la fonction à chacun des pipelines (maison et appartement) et on trie les résultats selon la MAE croissante (plus c’est bas, mieux c’est).

### 🏁 Sélection et sauvegarde des meilleurs modèles
On sélectionne le meilleur modèle pour chaque pipeline (celui avec la plus faible MAE) et on le sauvegarde dans un dossier dédié à Bordeaux

In [ ]:
# Liste pour stocker les résultats


def affiche_score(pipeline, X_bx, y_bx):
    results = []

    for name, model in pipeline.items():
        y_pred = model.predict(X_bx)
        
        mse = mean_squared_error(y_bx, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_bx, y_pred)
        r2 = r2_score(y_bx, y_pred)
        
        results.append({
            "Modele": name,
            "MSE": mse,
            "RMSE": rmse,
            "MAE": mae,
            "R2": r2
        })
    return pd.DataFrame(results)


# Optionnel : trier par MAE ou un autre critère
df_results_maison = affiche_score(pipeline_maison,X_bx_maisons, y_bx_maisons).sort_values(by="MAE")
df_results_appartement = affiche_score(pipeline_appartement, X_bx_apt, y_bx_apt).sort_values(by="MAE")

# Affichage final
print("pour les maison:")
print(df_results_maison)
print("pour les appartements:")
print(df_results_appartement)

meilleur_modele_maison = df_results_maison.iloc[0]["Modele"]
meilleur_modele_appartement = df_results_appartement.iloc[0]["Modele"]

# 💾 Sauvegarde des pipelines
os.makedirs("../app/models/Bordeaux", exist_ok=True)
joblib.dump(pipeline_maison[meilleur_modele_maison], "../app/models/Bordeaux/models_maison_Bordeaux.pkl")
joblib.dump(pipeline_appartement[meilleur_modele_appartement], "../app/models/Bordeaux/models_appartement_Bordeaux.pkl")

## 🔁 Comparaison des performances entre Lille et Bordeaux (optionnel)
Ce bloc de code permet de comparer les performances des modèles entraînés à Lille (2022) sur leurs propres données et sur celles de Bordeaux (2022).
Cela permet d’évaluer la stabilité et la capacité de généralisation des modèles en fonction de la ville.

>⚠️ Ce code est commenté par défaut.
Pour l’utiliser, il faut décommenter les lignes concernées (en retirant les # au début de chaque ligne).

### 📥 1. Chargement des résultats Lille
Les performances initiales des modèles sur Lille sont stockées dans un fichier JSON (resultats_modeles_maison_appartement.json). On les charge et on les transforme en DataFrames

### 🔎 2. Fonction de comparaison multi-métriques
Cette fonction fusionne les résultats entre Lille et Bordeaux par modèle, et calcule l’écart relatif (%) pour chaque métrique :

- RMSE_diff : écart relatif sur la RMSE
- MAE_diff : écart relatif sur la MAE
- R2_diff : écart relatif sur le R²
- écart_moyen_% : moyenne de ces écarts

### 📊 3. Résultats comparatifs
On affiche les résultats de comparaison pour les maisons et les appartements.

### 🏁 4. Sélection et sauvegarde du modèle le plus stable
On sélectionne le modèle avec l’écart moyen le plus faible (donc le plus stable entre les deux villes) et on le sauvegarde pour une utilisation API.

In [ ]:
# 1. Charger les résultats JSON de Lille
# with open("resultats_modeles_maison_appartement.json") as f:
#     results_lille = json.load(f)

# df_lille_maison = pd.DataFrame(results_lille["maison"])
# df_lille_appart = pd.DataFrame(results_lille["appartement"])

# def compare_models(df_lille, df_bordeaux, ville1="Lille", ville2="Bordeaux"):
#     df_merged = pd.merge(df_lille, df_bordeaux, on="Modele", suffixes=(f"_{ville1}", f"_{ville2}"))
    
#     metrics = ["RMSE", "MAE", "R2"]
#     for metric in metrics:
#         diff_col = f"{metric}_diff"
#         df_merged[diff_col] = (
#             abs(df_merged[f"{metric}_{ville1}"] - df_merged[f"{metric}_{ville2}"]) 
#             / abs(df_merged[f"{metric}_{ville2}"])
#         ) * 100

#     # Score global : moyenne des différences en %
#     df_merged["ecart_moyen_%"] = df_merged[[f"{m}_diff" for m in metrics]].mean(axis=1)

#     # Classement par stabilité
#     df_sorted = df_merged.sort_values("ecart_moyen_%")
    
#     return df_sorted[["Modele"] + [f"{m}_diff" for m in metrics] + ["ecart_moyen_%"]]

# # affichage
# df_result_maison = compare_models(df_lille_maison, df_results_maison)
# df_result_appartement = compare_models(df_lille_appart, df_results_appartement)

# print("comparatif écart Lille Bordeaux (maison)")
# display(df_result_maison)
# print("comparatif écart Lille Bordeaux (appartement)")
# display(df_result_appartement)

# meilleur_modele_maison = df_result_maison.iloc[0]["Modele"]
# meilleur_modele_appartement = df_result_appartement.iloc[0]["Modele"]

# # 💾 Sauvegarde des pipelines
# joblib.dump(pipeline_maison[meilleur_modele_maison], "../api/models/maison_models.pkl")
# joblib.dump(pipeline_appartement[meilleur_modele_appartement], "../api/models/appartement_models.pkl")